![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# ML Model Serialization in Redis

The `ModelStore` class below implements the following logic:
- Builds a model metadata index for model version management
- Handles model chunking, serialization, and deserialization to/from Redis using Pickle

Then we test with various Python ML-native data types and models.



In [ ]:
%pip install scikit-learn torch tensorflow

In [1]:
import os
import redis

from model_store import ModelStore

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

# Initialize Redis client
redis_client = redis.Redis.from_url(REDIS_URL)

In [2]:
# Initialize the ModelStore
model_store = ModelStore(redis_client, shard_size=1012*100)

15:27:15 redisvl.index.index INFO   Index already exists, not overwriting.


## Test with simple Scikit-Learn model

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load a simple dataset and train a RandomForest model
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy:.2f}")

Model accuracy: 1.00


In [4]:
# Save the trained model to Redis
model_name = "random_forest"
version = model_store.save_model(model, model_name)

2025-01-21 15:27:19.450 - model_store.store - INFO - Saving 'random_forest' model
2025-01-21 15:27:19.463 - model_store.store - INFO - Added model version record (0.0120s)
2025-01-21 15:27:19.464 - model_store.store - INFO - Starting model serialization and storage
2025-01-21 15:27:19.473 - model_store.store - INFO - Stored model in 2 shards (0.0091s)
2025-01-21 15:27:19.476 - model_store.store - INFO - Set shards (0.0021s)
2025-01-21 15:27:19.476 - model_store.store - INFO - Total save operation completed in 0.0265s


In [5]:
# Load the model from Redis
loaded_model = model_store.load_model(model_name)

# Verify that the loaded model works
y_pred_loaded = loaded_model.predict(X_test)
loaded_accuracy = accuracy_score(y_test, y_pred_loaded)
print(f"Loaded model accuracy: {loaded_accuracy:.2f}")

2025-01-21 15:27:21.193 - model_store.store - INFO - Loading 'random_forest' model
2025-01-21 15:27:21.197 - model_store.store - INFO - Retrieved model version metadata (0.0021s)
2025-01-21 15:27:21.197 - model_store.store - INFO - Starting model reconstruction from shards
2025-01-21 15:27:21.203 - model_store.store - INFO - Loaded model from 2 shards (0.0063s)
2025-01-21 15:27:21.204 - model_store.store - INFO - Load operation completed (0.0110s)


Loaded model accuracy: 1.00


## Test with 1Gb numpy array


In [6]:
import numpy as np

desired_size_bytes = 1024*1024*1024
num_elements = desired_size_bytes // 8
large_array = np.random.rand(num_elements).astype(np.float64)

model_name = "numpy_array"

In [7]:
version = model_store.save_model(large_array, model_name)

2025-01-21 15:27:28.351 - model_store.store - INFO - Saving 'numpy_array' model
2025-01-21 15:27:28.356 - model_store.store - INFO - Added model version record (0.0041s)
2025-01-21 15:27:28.356 - model_store.store - INFO - Starting model serialization and storage
2025-01-21 15:27:36.096 - model_store.store - INFO - Stored model in 10611 shards (7.7397s)
2025-01-21 15:27:36.103 - model_store.store - INFO - Set shards (0.0065s)
2025-01-21 15:27:36.103 - model_store.store - INFO - Total save operation completed in 7.7524s


In [8]:
# Load the model from Redis
loaded_model = model_store.load_model(model_name)

2025-01-21 15:27:46.092 - model_store.store - INFO - Loading 'numpy_array' model
2025-01-21 15:27:46.099 - model_store.store - INFO - Retrieved model version metadata (0.0062s)
2025-01-21 15:27:46.100 - model_store.store - INFO - Starting model reconstruction from shards
2025-01-21 15:27:49.886 - model_store.store - INFO - Loaded model from 10611 shards (3.7868s)
2025-01-21 15:27:49.927 - model_store.store - INFO - Load operation completed (3.8342s)


In [9]:
# check if all elements match in the array
sum(loaded_model == large_array) == num_elements

np.True_

## Test with pytorch model

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(1, 1)

    def forward(self, x):
        return self.fc(x)

# Create model, define loss and optimizer
model = SimpleModel()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Dummy data
x = torch.tensor([[1.0], [2.0], [3.0]], requires_grad=True)
y = torch.tensor([[2.0], [4.0], [6.0]])

# Training loop
for epoch in range(100):
    optimizer.zero_grad()
    output = model(x)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()

In [12]:
# Make a simple prediction
x_test = torch.tensor([[4.0]])
prediction = model(x_test).item()
print(f"Prediction for input 4.0: {prediction}")

Prediction for input 4.0: 6.903447151184082


In [15]:
# Save the trained model to Redis
model_name = "pytorch"
version = "1.0"
model_store.save_model(model, model_name, version=version)

2025-01-21 12:32:58.244 - model_store.store - INFO - Starting save operation for model 'pytorch'
2025-01-21 12:32:58.247 - model_store.store - INFO - Added version record (0.0028s)
2025-01-21 12:32:58.248 - model_store.store - INFO - Starting model serialization and storage
2025-01-21 12:32:58.250 - model_store.store - INFO - Stored model in 1 shards (0.0019s)
2025-01-21 12:32:58.252 - model_store.store - INFO - Set shards (0.0018s)
2025-01-21 12:32:58.252 - model_store.store - INFO - Total save operation completed in 0.0083s


'1.0'

In [16]:
# Load the model from Redis
loaded_model = model_store.load_model(model_name)

prediction = loaded_model(x_test).item()
print(f"Prediction for input 4.0 with loaded model: {prediction}")

2025-01-21 12:33:00.741 - model_store.store - INFO - Starting load operation for model 'pytorch'
2025-01-21 12:33:00.744 - model_store.store - INFO - Retrieved version metadata (0.0018s)
2025-01-21 12:33:00.745 - model_store.store - INFO - Starting model reconstruction from shards
2025-01-21 12:33:00.747 - model_store.store - INFO - Loaded model from 1 shards (0.0019s)
2025-01-21 12:33:00.747 - model_store.store - INFO - Total load operation completed (0.0055s)


Prediction for input 4.0 with loaded model: 6.903447151184082


## Test with tensorflow

In [17]:
import tensorflow as tf

# Define a simple model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=(1,))
])

# Compile the model
model.compile(optimizer='sgd', loss='mse')

# Dummy data
x = tf.constant([[1.0], [2.0], [3.0]])
y = tf.constant([[2.0], [4.0], [6.0]])

# Train the model
model.fit(x, y, epochs=100, verbose=0)

/Users/tyler.hutcherson/Library/Caches/pypoetry/virtualenvs/redis-model-store-__eJJx5C-py3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
# Make a simple prediction
x_test = tf.constant([[4.0]])
prediction = model(x_test).numpy()[0, 0]
print(f"Prediction for input 4.0: {prediction}")

Prediction for input 4.0: 7.460770606994629


In [19]:
# Save the trained model to Redis
model_name = "tensorflow"
version = "1.0"
model_store.save_model(model, model_name, version=version)

2025-01-21 12:33:24.276 - model_store.store - INFO - Starting save operation for model 'tensorflow'
2025-01-21 12:33:24.280 - model_store.store - INFO - Added version record (0.0028s)
2025-01-21 12:33:24.280 - model_store.store - INFO - Starting model serialization and storage
2025-01-21 12:33:24.291 - model_store.store - INFO - Stored model in 1 shards (0.0107s)
2025-01-21 12:33:24.292 - model_store.store - INFO - Set shards (0.0014s)
2025-01-21 12:33:24.293 - model_store.store - INFO - Total save operation completed in 0.0163s


'1.0'

In [20]:
# Load the model from Redis
loaded_model = model_store.load_model(model_name)

prediction = loaded_model(x_test).numpy()[0, 0]
print(f"Prediction for input 4.0 with loaded model: {prediction}")

2025-01-21 12:33:24.297 - model_store.store - INFO - Starting load operation for model 'tensorflow'
2025-01-21 12:33:24.300 - model_store.store - INFO - Retrieved version metadata (0.0027s)
2025-01-21 12:33:24.301 - model_store.store - INFO - Starting model reconstruction from shards
2025-01-21 12:33:24.312 - model_store.store - INFO - Loaded model from 1 shards (0.0108s)
2025-01-21 12:33:24.312 - model_store.store - INFO - Total load operation completed (0.0150s)


Prediction for input 4.0 with loaded model: 7.460770606994629


# Clear the model store